In [20]:
from pathlib import Path
from functools import partial

import pybem as pb
import numpy as np
import pandas as pd

In [17]:
PATH_POLARS = Path("../polars")

In [29]:
csv_reader = partial(pd.read_csv, sep=";", index_col=None)


def load_polar(airfoil_type, which="cl"):
    """Loads the airfoil polars
    
    Parameters
    ----------
    airfoil_type : str
    which : str
        "cl" or "cd"
    
    Returns
    -------
    np.array N x 2
    """

    _path_lift = PATH_POLARS / airfoil_type / "lift.csv"
    _path_drag = PATH_POLARS / airfoil_type / "drag.csv"

    polar_cl = csv_reader(_path_lift).values
    polar_cd = csv_reader(_path_drag).values

    polars = {"cl": polar_cl, "cd": polar_cd}

    return polars[which]

In [26]:
# Define known sections
sections = [
    pb.Section(
        name="Hub",
        r=0.3,
        beta=60,
        chord=0.4,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="765", which="cl"),
            polar_cd=load_polar(airfoil_type="765", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 0",
        r=0.3,
        beta=60,
        chord=0.4,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="765", which="cl"),
            polar_cd=load_polar(airfoil_type="765", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 1",
        r=0.6,
        beta=45,
        chord=0.35,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="764", which="cl"),
            polar_cd=load_polar(airfoil_type="764", which="cd"),
        ),
    ),
    pb.Section(
        name="Station 2",
        r=0.6,
        beta=45,
        chord=0.35,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="763", which="cl"),
            polar_cd=load_polar(airfoil_type="763", which="cd"),
        ),
    ),
    pb.Section(
        name="Tip",
        r=1.2,
        beta=30,
        chord=0.2,
        airfoil=pb.Airfoil(
            polar_cl=load_polar(airfoil_type="762", which="cl"),
            polar_cd=load_polar(airfoil_type="762", which="cd"),
        ),
    ),
]

# Define propeller
B = 6
propeller = pb.Propeller(B=B, sections=sections)

# Define flow conditions and BEM method
J = 0.2
bem = pb.BladeElementMethod(J=J, propeller=propeller, tip_loss=False, hub_loss=False)

# Solve
bem.solve()

# Compute forces
CT, CQ = bem.integrate_forces()